# imports

In [3]:
# imports
import pandas as pd

import numpy as np
from sklearn.pipeline import Pipeline

from src.utils import utils
from src.data_processing import preprocessing as prep
import logging

%load_ext autoreload
%autoreload 2

# Read Data

In [4]:
root_dir = utils.get_proj_root()
raw_data =  pd.read_csv(root_dir.joinpath('data/processed/preprocessed.csv'))
raw_data.head()

,industry,symbol,year,currentRatio,currentRatio_percentage_change,quickRatio,quickRatio_percentage_change,daysOfSalesOutstanding,daysOfSalesOutstanding_percentage_change,daysOfInventoryOutstanding,...,priceEarningsToGrowthRatio_percentage_change,dividendYield,dividendYield_percentage_change,enterpriseValueMultiple,enterpriseValueMultiple_percentage_change,interestRate,interestRate_percentage_change,adjDividend,dps_growth,dps_change_next_year
0,24,291,2012.0,2.198387,-2.275946,1.385806,-0.090461,49.567449,3.987852,89.289449,...,6.939145,0.025377,-5.500411,8.692139,5.900833,0.14,40.000000,2.36,7.272727,0
1,24,291,2013.0,1.698186,-22.753086,1.012270,-26.954450,50.284895,1.447414,87.567366,...,21.913077,0.018089,-28.717392,12.326051,41.806877,0.11,-21.428571,2.54,7.627119,0
2,24,291,2014.0,1.961487,15.504836,1.127209,11.354593,48.611609,-3.327613,82.245394,...,-39.851336,0.020773,14.836058,13.065263,5.997151,0.09,-18.181818,3.42,34.645669,0
3,24,291,2015.0,1.543411,-21.314241,0.852768,-24.346986,50.082909,3.026645,83.473315,...,799.567847,0.027175,30.819267,12.318266,-5.717433,0.13,44.444444,4.10,19.883041,0
4,24,291,2016.0,1.885512,22.165260,1.136839,33.311664,53.242552,6.308824,82.149269,...,-84.149916,0.024801,-8.738086,13.479738,9.428861,0.40,207.692308,4.44,8.292683,0


# Data Augmentation

## data imbalance

In [5]:
df = raw_data.copy()

# chekc data imbalance
label_col_name = 'dps_change_next_year'
df[label_col_name].value_counts()

dps_change_next_year
0    4303
1     264
Name: count, dtype: int64

In [6]:
df.isna().sum()

industry                          0
symbol                            0
year                              0
currentRatio                      0
currentRatio_percentage_change    0
                                 ..
interestRate                      0
interestRate_percentage_change    0
adjDividend                       0
dps_growth                        0
dps_change_next_year              0
Length: 89, dtype: int64

In [7]:
# 


training_data = df.loc[df['year'] != 2021]
testing_data = df.loc[df['year'] == 2021]

X_train = training_data.drop(label_col_name, axis="columns")
y_train = training_data[label_col_name]
X_test = testing_data.drop(label_col_name, axis="columns")
y_test = testing_data[label_col_name]

In [8]:
from imblearn.over_sampling import SMOTE

def balance_data(X, y, random_state=None):

    smote = SMOTE(random_state=random_state)
    X_resample, y_resample = smote.fit_resample(X, y)


    return X_resample, y_resample

X_resample, y_resample = balance_data(X_train, y_train)
y_train.value_counts().to_dict()
y_resample.value_counts().to_dict()

{0: 3838, 1: 3838}

In [14]:
X_resample[label_col_name] = y_resample